# Import Libraries and Data

In [1]:
import pandas as pd
import re

import numpy as np
from functools import reduce

from scipy.stats import linregress
from sklearn.preprocessing import StandardScaler

In [2]:
DATA_PATH = '../data'

In [3]:
cons = pd.read_parquet(f'{DATA_PATH}/raw/q2_consDF_final.pqt')
acct = pd.read_parquet(f'{DATA_PATH}/raw/q2_acctDF_final.pqt')
inflows = pd.read_parquet(f'{DATA_PATH}/raw/q2_inflows_final.pqt')
outflows = pd.read_parquet(f'{DATA_PATH}/raw/q2_outflows_final.pqt')

# Create Features

In [4]:
features = []

### Balance Summary Statistics at time of evaluation
 - minbalance: The maximum balance a consumer has in any of their accounts
 - maxbalance: The maximum balance a consumer has in any of their accounts
 - stdbalance: The standard deviation of all balances of a consumer's account

In [5]:
def get_balance_stats(acct):
    balance_var = acct[['prism_consumer_id','balance','balance_date']].groupby('prism_consumer_id').agg({
        'balance':['min', 'max', 'std'],
        'balance_date':['max']
    }).reset_index()
    balance_var.fillna(0.0, inplace=True)

    balance_var.columns = balance_var.columns.to_flat_index().map(lambda x: x[1] + x[0])
    return balance_var

balance_var = get_balance_stats(acct)
features.append(balance_var)
balance_var.head()

,prism_consumer_id,minbalance,maxbalance,stdbalance,maxbalance_date
0,0,6182.60,9907.23,2633.711130,2023-04-13
1,2,8079.43,17426.83,6609.609926,2022-02-15
2,4,0.00,4871.39,2644.903088,2021-08-13
3,7,517.95,7071.18,3766.231217,2021-08-08
4,9,13348.76,13348.76,0.000000,2023-04-19


### Disposable Income
How much disposable income does a consumer have available after looking at all purchases and income sources in the dataset

In [6]:
cons_eval_dates = cons.sort_values('evaluation_date')[['prism_consumer_id','evaluation_date']]
outflow_merged = pd.merge(outflows, cons_eval_dates, on="prism_consumer_id", how="left")
inflow_merged = pd.merge(inflows, cons_eval_dates, on="prism_consumer_id", how="left")
outflow_merged['amount'] = outflow_merged['amount'] * -1
total_cashflow = pd.concat([inflow_merged, outflow_merged])
total_cashflow = total_cashflow[total_cashflow['posted_date'] <= total_cashflow['evaluation_date']]

In [7]:
def get_disposable_income(total_cashflow):
    total_balance = (
        total_cashflow[[
            'prism_consumer_id',
            'amount',
            'posted_date',
        ]]
        .groupby('prism_consumer_id')
        .agg({
            'amount':['sum'],
            'posted_date':['min','max']
        })
    )

    total_balance['date_range'] = pd.to_timedelta(total_balance['posted_date','max'] - total_balance['posted_date','min']).dt.days / 365
    total_balance = pd.DataFrame({
        'total_balance': total_balance['amount','sum'] / total_balance['date_range'], 
        'total_balance_date': total_balance['posted_date', 'max']
    }).reset_index()
    return total_balance

total_balance = get_disposable_income(total_cashflow)
features.append(total_balance)
total_balance.head()

,prism_consumer_id,total_balance,total_balance_date
0,0,15154.285773,2023-04-12
1,2,1546.065942,2022-02-14
2,4,3377.081389,2021-08-12
3,7,7311.445429,2021-08-06
4,9,5873.815289,2023-04-18


### Monthly Balance Statistics
Features that look at average monthly activity:
 - Average Monthly Spending
 - Average number of purchases made per month

In [8]:
def get_monthly_balance_stats(outflow_merged):
    outflow_valid = outflow_merged[outflow_merged['posted_date'] <= outflow_merged['evaluation_date']]
    outflow_counts = outflow_valid.groupby('prism_consumer_id').agg({
        'amount':['count','sum'],
        'posted_date':['min','max'],
    }).reset_index()

    outflow_counts['date_range'] = pd.to_timedelta(outflow_counts['posted_date','max'] - outflow_counts['posted_date','min']).dt.days / 365 * 12

    outflow_counts = pd.DataFrame({
        'prism_consumer_id': outflow_counts['prism_consumer_id'],
        'avg_monthly_spending': -1 * outflow_counts['amount','sum'] / outflow_counts['date_range'],
        'num_monthly_purchase': outflow_counts['amount','count'] / outflow_counts['date_range'],
        # 'monthly_purchase_date': outflow_counts['posted_date','max'],
    })

    outflow_counts.replace([np.inf, -np.inf], 0.0, inplace=True)
    return outflow_counts

outflow_counts = get_monthly_balance_stats(outflow_merged)
features.append(outflow_counts)
outflow_counts.head()

,prism_consumer_id,avg_monthly_spending,num_monthly_purchase
0,0,7703.614883,80.915055
1,2,9354.337258,98.743961
2,4,30563.426238,33.035880
3,7,23188.278116,138.265235
4,9,11186.144777,79.267677


### Savings Feature
A count of how many times someone has pulled from savings account


In [9]:
def get_num_savings_transfer(inflows):
    transfer_from_savings = inflows[inflows['category_description']=='SELF_TRANSFER']
    transfer_from_savings = transfer_from_savings[transfer_from_savings['memo_clean'].str.contains('Savings')]
    count_tfs = transfer_from_savings.groupby('prism_consumer_id').count().reset_index()
    inflow_ids = pd.merge(inflows[['prism_consumer_id']], count_tfs, on='prism_consumer_id', how='left')
    inflow_ids = inflow_ids.fillna(0).drop_duplicates(subset=['prism_consumer_id']).reset_index()[['prism_consumer_id', 'memo_clean']]

    return inflow_ids

inflow_ids = get_num_savings_transfer(inflows)
features.append(inflow_ids)
inflow_ids

,prism_consumer_id,memo_clean
0,0,0.0
1,2,0.0
2,4,0.0
3,7,0.0
4,9,0.0
...,...,...
2973,3991,63.0
2974,3992,0.0
2975,3993,22.0
2976,3996,241.0


### Unsufficient Funds
A boolean output for whether a consumer has an account that is negative or near 0.

In [10]:
def get_unsufficient_funds(acct):
    acct = acct.copy()
    acct['unsufficient_balance'] = acct['balance'].apply(lambda x: x <= 1)
    unsufficient_accts = acct.groupby('prism_consumer_id')['unsufficient_balance'].count().reset_index()[['prism_consumer_id', 'unsufficient_balance']]
    return unsufficient_accts

unsufficient_accts = get_unsufficient_funds(acct)
features.append(unsufficient_accts)
unsufficient_accts.head()


,prism_consumer_id,unsufficient_balance
0,0,2
1,2,2
2,4,3
3,7,3
4,9,1


### Amount of Accounts Per User
A number that represents the total number of accounts each consumer has.

In [11]:
def get_num_accounts(acct):
    acct = acct.rename(columns = {'prism_consumer_id':'grouping_prism_consumer_id'})
    account_count = acct.groupby('grouping_prism_consumer_id')[['grouping_prism_consumer_id']].count()
    account_count = account_count.rename(columns = {'grouping_prism_consumer_id': 'account_count'}).reset_index()
    account_count = account_count.rename(columns = {'grouping_prism_consumer_id': 'prism_consumer_id'})
    return account_count

account_count = get_num_accounts(acct)
features.append(account_count)
account_count.head()

,prism_consumer_id,account_count
0,0,2
1,2,2
2,4,3
3,7,3
4,9,1


### Category_Monthly_Slope
Calculate the monthly spending slope for each unique combination of 'category_description' and 'prism_consumer_id' in a dataset.

In [12]:
# Create 'month' and 'year' Columns
outflows['month'] = pd.to_datetime(outflows['posted_date']).dt.month
outflows['year'] = pd.to_datetime(outflows['posted_date']).dt.year

In [13]:
# Group Data by Categories and Consumers
cat_group = outflows.groupby([ 'category_description', 'prism_consumer_id','year', 'month'])['amount'].sum().reset_index()
# cat_group.head()

In [14]:
# Calculate Slopes

merged_df = pd.DataFrame()

for i in cat_group['category_description'].unique():
    cat_name = str(i)
    id_5949 = cat_group[cat_group['category_description']== i]
    slopes = []
    for i in id_5949['prism_consumer_id'].unique():
        cat_id = id_5949[id_5949['prism_consumer_id']==i]
        slope, intercept, _, _, _ = linregress(range(len(cat_id)), cat_id['amount'])
        slopes.append(slope)
        
    df = pd.DataFrame({'prism_consumer_id': id_5949['prism_consumer_id'].unique(), cat_name + '_slp': slopes})
    
    if merged_df.empty:
        merged_df = df.copy()
    else:
        merged_df = pd.merge(merged_df, df, on='prism_consumer_id', how='outer')
        
#     print(df)

/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/Users/darrenjiang/opt/anaconda3/envs/dsc180/lib/python3.9/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered

In [15]:
features.append(merged_df)
merged_df.head()

,prism_consumer_id,ACCOUNT_FEES_slp,ATM_CASH_slp,AUTOMOTIVE_slp,BILLS_UTILITIES_slp,BNPL_slp,CHILD_DEPENDENTS_slp,CREDIT_CARD_PAYMENT_slp,EDUCATION_slp,ENTERTAINMENT_slp,...,LOAN_slp,MORTGAGE_slp,OVERDRAFT_slp,PAYCHECK_slp,PETS_slp,RENT_slp,SELF_TRANSFER_slp,TAX_slp,TRAVEL_slp,UNCATEGORIZED_slp
0,0,0.870000,225.750000,-7.157637,NaN,NaN,NaN,-76.308392,NaN,-1.432622,...,40.927912,NaN,NaN,NaN,61.000000,NaN,102.802198,NaN,-6.291071,51.136455
1,4,-17.000000,NaN,119.622000,NaN,NaN,NaN,1545.580220,NaN,NaN,...,42.243182,NaN,NaN,NaN,NaN,NaN,76.651703,NaN,NaN,69.363167
2,7,-1.693531,222.096868,-3.399286,NaN,0.0,NaN,17.425110,NaN,-4.675490,...,-2.386469,NaN,-10.5,NaN,2.866000,NaN,NaN,-2.183167,193.235152,122.932582
3,9,NaN,40.000000,4.291758,NaN,NaN,NaN,148.666888,NaN,30.675000,...,-2.659021,NaN,NaN,NaN,9.620818,714.528,-19.857143,NaN,5.010000,327.411786
4,12,-0.563273,-467.150385,159.907343,NaN,NaN,NaN,-96.061099,NaN,-0.772692,...,-37.410440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.842182,-106.031538


### Non-Essential Ratio
The Non-Essential Ratio is a metric that represents the proportion of spending deemed non-essential relative to the total spending for a given consumer. This ratio is calculated by dividing the total spending in categories considered non-essential by the overall spending, including both essential and non-essential categories.

In [16]:
# a list of spending categories considered essential for each consumer.
essential_lst = [
    'ACCOUNT_FEES',
    'GROCERIES','LOAN',
    'CREDIT_CARD_PAYMENT',
    'BILLS_UTILITIES',
    'AUTOMOTIVE',
    'INSURANCE',
    'ESSENTIAL_SERVICES',
    'HEALTHCARE_MEDICAL',
    'PETS','TAX','RENT', 
    'MORTGAGE', 'CHILD_DEPENDENTS'
]

In [17]:
# non-essential spending for each consumer based on the defined essential categories.
def non_essential_ratio(essential_lst, outflows):
    
    # Create a binary column indicating if the category is essential
    outflows = outflows.copy()
    outflows['essential'] = outflows['category_description'].isin(essential_lst).astype(int)
    
    # Calculate the non-essential ratio for each prism_consumer_id
    result = (
        outflows.groupby(['prism_consumer_id', 'essential'])['amount'].sum()
        .unstack(fill_value=0)
        .assign(non_ess_ratio=lambda x: x[0] / x.sum(axis=1))
        .reset_index()
    )
    
    id_lst = result['prism_consumer_id'].tolist()
    ess_ratio = result['non_ess_ratio'].tolist()
    df = pd.DataFrame({'prism_consumer_id': id_lst, 'non_ess_ratio': ess_ratio})
    
    return df


In [18]:
non_essential = non_essential_ratio(essential_lst, outflows)
features.append(non_essential)
non_essential.head()

,prism_consumer_id,non_ess_ratio
0,0,0.581717
1,2,0.509382
2,4,0.280896
3,7,0.646896
4,9,0.417012


### Standardized Balance Slope
Calculates the balance slope for each consumer by examining their monthly balances using the linregress method.

In [19]:
# Calculate the standardized balance slope for each consumer
def stdzd_bal_slp(total_cashflow, acct):    
    
    total_c = total_cashflow.copy()
    
    # Extract the month and year from the posted_date column
    total_c['month'] = pd.to_datetime(total_c['posted_date']).dt.month
    total_c['year'] = pd.to_datetime(total_c['posted_date']).dt.year
    
    # Group by consumer, year, and month to sum the amounts
    total_c = total_c.groupby(['prism_consumer_id', 'year', 'month'])['amount'].sum().reset_index()
    
    # Get unique consumer IDs
    unique_ids = total_c['prism_consumer_id'].unique()
    
    # Initialize an empty list to store calculated slopes
    slopes = []
    
    # Loop through unique consumer IDs
    for i in unique_ids:
        # Filter data for the current consumer ID
        id_ = total_c[total_c['prism_consumer_id'] == i]
        
        # Get the initial balance for the consumer
        start_point = acct[acct['prism_consumer_id'] == i]['balance'].sum()
        
        # Reverse the monthly amounts and calculate cumulative balance
        diff_reversed = id_['amount'].iloc[::-1].tolist()
        line = np.cumsum(np.concatenate([[start_point], -np.array(diff_reversed)]))[::-1]
        
        # Use linregress to find the slope of the cumulative balance
        slope, intercept, _, _, _ = linregress(np.arange(len(line)), line)
        
        # Append the calculated slope to the list
        slopes.append(slope)
    
    # Create a DataFrame with consumer IDs and corresponding slopes
    df = pd.DataFrame({'prism_consumer_id': unique_ids, 'stdzd_bal_slp': slopes})
    
    # Standardize the slope values using StandardScaler
    features = df[['stdzd_bal_slp']]
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    scaled_df = pd.DataFrame(scaled_features, columns=features.columns)    
  
    return pd.concat([df[['prism_consumer_id']], scaled_df], axis=1)


In [20]:
slop = stdzd_bal_slp(total_cashflow, acct)
features.append(slop)
slop.head()

,prism_consumer_id,stdzd_bal_slp
0,0,0.374371
1,2,0.007861
2,4,0.540605
3,7,0.346054
4,9,0.195140


### Positive Remaining Ratio
Determines the amount of months where cashflow is positive out of all months in the dataset for each consumer

In [21]:
def get_PRR(total_cashflow):
    # Get month and year of inflow and outflow
    total_cashflow = total_cashflow.copy()
    total_cashflow['month'] = pd.to_datetime(total_cashflow['posted_date']).dt.month
    total_cashflow['year'] = pd.to_datetime(total_cashflow['posted_date']).dt.year
    # Group transactions by month and calculate remaining balance on each month for each client
    total_cashflow = total_cashflow.groupby(['prism_consumer_id', 'year', 'month'])['amount'].sum().reset_index()
    # Determine if the remaining amount is positive for each month
    total_cashflow['PRR'] = (total_cashflow['amount'] > 0).astype(int)
    # Calculate the ratio of months with positive remaining amounts
    total_cashflow = total_cashflow.groupby('prism_consumer_id')['PRR'].mean().reset_index()
    return total_cashflow

In [22]:
PRR = get_PRR(total_cashflow)
features.append(PRR)
PRR.head()

,prism_consumer_id,PRR
0,0,0.538462
1,2,0.333333
2,4,0.461538
3,7,0.384615
4,9,0.538462


### Credit Ratio
Determines the maximum number of consecutive months in which a consumer pays of a loan

In [23]:
def get_CR(outflows):
    # Get month range
    outflow_transactions = outflows.copy()
    outflow_transactions['month'] = pd.to_datetime(outflow_transactions['posted_date']).dt.month
    outflow_transactions['year'] = pd.to_datetime(outflow_transactions['posted_date']).dt.year
    outflow_transactions['date_id'] = 12 * (outflow_transactions['year'] - min(outflow_transactions['year'])) + outflow_transactions['month']
    # Get number of transactions for each category for each consumer
    cat_groups = outflow_transactions.groupby(['prism_consumer_id', 'date_id', 'category_description']).size().reset_index(name='count')
    # Find number of months where consumer spent money on credit card payments or loans
    cat_counts = cat_groups.groupby(['prism_consumer_id','category_description']).agg({'count':'count', 'date_id':['min', 'max']}).reset_index()
    credit_counts = cat_counts[np.isin(cat_counts['category_description'], ['LOAN', 'CREDIT_CARD_PAYMENT'])].copy()
    # Get maximum credit ratio
    credit_counts['CR'] = credit_counts['count','count'] / (credit_counts['date_id','max'] - credit_counts['date_id','min'] + 1)
    return credit_counts.groupby('prism_consumer_id')['CR'].max().reset_index()

In [24]:
CR = get_CR(outflows)
features.append(CR)
CR.head()

,prism_consumer_id,CR
0,0,1.0
1,2,1.0
2,4,1.0
3,7,1.0
4,9,1.0


### Total Category Summary Statistics
 - get mean, median, min, and max of spending in each category throughout dataset

In [25]:
def get_cat_stats(outflows):
    cat_aggs = (
        outflows
        .groupby(['prism_consumer_id', 'category_description'])['amount']
        .agg(['mean', 'median', 'min', 'max'])
        .reset_index()
        .pivot_table(
            index = 'prism_consumer_id',
            columns = 'category_description',
            values = ['mean', 'median', 'min', 'max']
        )
        .reset_index()
    )

    cat_aggs.columns = ["".join(a) for a in cat_aggs.columns.to_flat_index()]
    return cat_aggs

cat_stats = get_cat_stats(outflows)
features.append(cat_stats)
cat_stats.head()

,prism_consumer_id,maxACCOUNT_FEES,maxATM_CASH,maxAUTOMOTIVE,maxBILLS_UTILITIES,maxBNPL,maxCHILD_DEPENDENTS,maxCREDIT_CARD_PAYMENT,maxEDUCATION,maxENTERTAINMENT,...,minLOAN,minMORTGAGE,minOVERDRAFT,minPAYCHECK,minPETS,minRENT,minSELF_TRANSFER,minTAX,minTRAVEL,minUNCATEGORIZED
0,0,3.00,503.00,101.58,NaN,NaN,NaN,500.00,NaN,83.45,...,1.00,NaN,NaN,NaN,19.00,NaN,25.0,NaN,4.0,3.21
1,2,NaN,1000.00,75.87,NaN,NaN,NaN,201.13,NaN,91.61,...,426.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25
2,4,25.00,2466.11,299.84,NaN,NaN,NaN,25000.00,NaN,250.00,...,262.62,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,91.00
3,7,237.06,985.00,331.54,NaN,382.31,NaN,3000.00,NaN,1031.95,...,35.50,NaN,35.0,NaN,50.78,NaN,NaN,100.0,3.0,1.00
4,9,3.00,500.00,180.00,NaN,NaN,NaN,2830.46,NaN,245.63,...,70.98,NaN,NaN,NaN,17.92,2375.24,40.0,NaN,3.0,9.75


### Monthly Category Aggregates
- avg_monthly_cat_num_trans: get the average number of outflow transactions a consumer makes per month
- avg_monthly_cat_spending: get the average amount a consumer makes per month

In [26]:
def avg_monthly_cat_num_trans(outflows):
    # Get month range
    outflow_transactions = outflows.copy()
    outflow_transactions['month'] = pd.to_datetime(outflow_transactions['posted_date']).dt.month
    outflow_transactions['year'] = pd.to_datetime(outflow_transactions['posted_date']).dt.year
    outflow_transactions['date_id'] = 12 * (outflow_transactions['year'] - min(outflow_transactions['year'])) + outflow_transactions['month']
    # Get average number of transaction per month
    return (
        outflow_transactions
        .groupby(['prism_consumer_id', 'category_description', 'date_id'])['amount']
        .count()
        .reset_index(name='count')
        .groupby(['prism_consumer_id', 'category_description'])
        .mean()
        .reset_index()
        .pivot(index='prism_consumer_id', columns='category_description', values='count')
        .add_suffix('_count')
        .reset_index()
        .rename_axis(None, axis=1)
    )
avg_month_transaction_count = avg_monthly_cat_num_trans(outflows)
features.append(avg_month_transaction_count)
avg_month_transaction_count.head()

,prism_consumer_id,ACCOUNT_FEES_count,ATM_CASH_count,AUTOMOTIVE_count,BILLS_UTILITIES_count,BNPL_count,CHILD_DEPENDENTS_count,CREDIT_CARD_PAYMENT_count,EDUCATION_count,ENTERTAINMENT_count,...,LOAN_count,MORTGAGE_count,OVERDRAFT_count,PAYCHECK_count,PETS_count,RENT_count,SELF_TRANSFER_count,TAX_count,TRAVEL_count,UNCATEGORIZED_count
0,0,1.666667,1.000000,3.615385,NaN,NaN,NaN,5.000000,NaN,1.833333,...,4.076923,NaN,NaN,NaN,1.500000,NaN,5.230769,NaN,1.285714,2.363636
1,2,NaN,4.000000,7.000000,NaN,NaN,NaN,4.000000,NaN,4.500000,...,2.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.666667
2,4,1.000000,1.000000,1.800000,NaN,NaN,NaN,10.000000,NaN,1.000000,...,2.333333,NaN,NaN,NaN,NaN,NaN,4.846154,NaN,NaN,2.000000
3,7,6.416667,7.230769,2.615385,NaN,1.333333,NaN,14.769231,NaN,3.083333,...,7.500000,NaN,1.75,NaN,1.000000,NaN,NaN,1.222222,3.700000,18.769231
4,9,3.000000,1.818182,5.538462,NaN,NaN,NaN,3.333333,NaN,2.200000,...,1.000000,NaN,NaN,NaN,1.636364,1.25,1.166667,NaN,1.000000,1.875000


In [27]:
def avg_monthly_cat_spending(outflows):
    # Get month range
    outflow_transactions = outflows.copy()
    outflow_transactions['month'] = pd.to_datetime(outflow_transactions['posted_date']).dt.month
    outflow_transactions['year'] = pd.to_datetime(outflow_transactions['posted_date']).dt.year
    outflow_transactions['date_id'] = 12 * (outflow_transactions['year'] - min(outflow_transactions['year'])) + outflow_transactions['month']
    # Get average amount of transaction per month
    return (
        outflow_transactions
        .groupby(['prism_consumer_id', 'category_description', 'date_id'])['amount']
        .mean()
        .reset_index(name='mean_transaction')
        .groupby(['prism_consumer_id', 'category_description'])
        .mean()
        .reset_index()
        .pivot(index='prism_consumer_id', columns='category_description', values='mean_transaction')
        .add_suffix('_mean')
        .reset_index()
        .rename_axis(None, axis=1)
    )
avg_month_spending = avg_monthly_cat_spending(outflows)
features.append(avg_month_spending)
avg_month_spending.head()

,prism_consumer_id,ACCOUNT_FEES_mean,ATM_CASH_mean,AUTOMOTIVE_mean,BILLS_UTILITIES_mean,BNPL_mean,CHILD_DEPENDENTS_mean,CREDIT_CARD_PAYMENT_mean,EDUCATION_mean,ENTERTAINMENT_mean,...,LOAN_mean,MORTGAGE_mean,OVERDRAFT_mean,PAYCHECK_mean,PETS_mean,RENT_mean,SELF_TRANSFER_mean,TAX_mean,TRAVEL_mean,UNCATEGORIZED_mean
0,0,2.140000,252.500000,30.155222,NaN,NaN,NaN,88.014699,NaN,16.102847,...,372.020218,NaN,NaN,NaN,54.250000,NaN,185.239927,NaN,36.380714,120.534303
1,2,NaN,280.000000,34.319808,NaN,NaN,NaN,125.292389,NaN,33.082000,...,668.712222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.128019
2,4,16.500000,2466.110000,276.491500,NaN,NaN,NaN,1619.751598,NaN,250.000000,...,537.299361,NaN,NaN,NaN,NaN,NaN,467.658890,NaN,NaN,318.738704
3,7,38.318027,376.691127,67.979679,NaN,93.301667,NaN,227.373716,NaN,169.973554,...,346.317099,NaN,35.0,NaN,93.585000,NaN,NaN,395.320556,133.658533,295.393790
4,9,3.000000,133.266591,98.039610,NaN,NaN,NaN,1083.478194,NaN,75.057667,...,89.020000,NaN,NaN,NaN,60.401591,2376.055,109.166667,NaN,5.505000,1085.457917


### Proportion of Spending

In [28]:
def total_prop_spending(outflows):
    cons_cat_spending = outflows.groupby(['prism_consumer_id', 'category_description'])['amount'].sum().reset_index()
    cons_total = outflows.groupby(['prism_consumer_id'])['amount'].sum().reset_index()

    cat_prop_indiv = cons_cat_spending.merge(cons_total, on='prism_consumer_id')
    cat_prop_indiv['prop'] = cat_prop_indiv['amount_x'] / cat_prop_indiv['amount_y']
    cat_prop_indiv = cat_prop_indiv[['prism_consumer_id', 'category_description', 'prop']]

    return (
        cat_prop_indiv.pivot(index='prism_consumer_id', columns='category_description', values='prop')
        .add_suffix('_prop')
        .reset_index()
        .rename_axis(None, axis=1)
        .fillna(0.0)
    )

total_cons_spending_prop = total_prop_spending(outflows)
features.append(total_cons_spending_prop)
total_cons_spending_prop.head()

,prism_consumer_id,ACCOUNT_FEES_prop,ATM_CASH_prop,AUTOMOTIVE_prop,BILLS_UTILITIES_prop,BNPL_prop,CHILD_DEPENDENTS_prop,CREDIT_CARD_PAYMENT_prop,EDUCATION_prop,ENTERTAINMENT_prop,...,LOAN_prop,MORTGAGE_prop,OVERDRAFT_prop,PAYCHECK_prop,PETS_prop,RENT_prop,SELF_TRANSFER_prop,TAX_prop,TRAVEL_prop,UNCATEGORIZED_prop
0,0,0.000079,0.008262,0.016674,0.0,0.000000,0.0,0.063996,0.0,0.003748,...,0.198970,0.0,0.00000,0.0,0.001800,0.00000,0.169256,0.000000,0.004219,0.031668
1,2,0.000000,0.136191,0.034459,0.0,0.000000,0.0,0.071576,0.0,0.013950,...,0.218708,0.0,0.00000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.096162
2,4,0.000091,0.006817,0.006199,0.0,0.000000,0.0,0.642766,0.0,0.000691,...,0.051090,0.0,0.00000,0.0,0.000000,0.00000,0.073396,0.000000,0.000000,0.015665
3,7,0.008822,0.129482,0.006069,0.0,0.001776,0.0,0.169520,0.0,0.027906,...,0.129166,0.0,0.00089,0.0,0.001360,0.00000,0.000000,0.013673,0.021760,0.224591
4,9,0.000067,0.019953,0.050651,0.0,0.000000,0.0,0.323585,0.0,0.005228,...,0.008002,0.0,0.00000,0.0,0.008849,0.08901,0.005356,0.000000,0.000082,0.111047


### Overdraft Frequency

Marks users that have more than 1 monthly overdraft transaction

In [29]:
# overdraft frequency, high risk = more than once a month
overdraft = outflows[outflows['category_description'] == 'OVERDRAFT']
crosstab = pd.crosstab(overdraft['posted_date'], overdraft['prism_consumer_id'])
crosstab.head()

prism_consumer_id,7,11,56,78,81,100,106,153,163,180,...,5863,5871,5872,5877,5883,5888,5908,5928,5935,5949
posted_date,,,,,,,,,,,,,,,,,,,,,
2018-07-20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
outflow_count = pd.DataFrame(crosstab[crosstab > 1].stack()).reset_index()
outflow_count = outflow_count.rename(columns = {'0': 'Overdraft Occurance'})
outflow_count_index = pd.DataFrame(outflow_count.groupby('prism_consumer_id').count()['posted_date']).index
outflow_count_index

Index([   7,   11,   56,   78,   81,  100,  163,  200,  319,  418,
       ...
       5679, 5681, 5708, 5709, 5732, 5775, 5794, 5812, 5888, 5908],
      dtype='int64', name='prism_consumer_id', length=132)

In [31]:
outflows['Overdraft Count'] = outflows['prism_consumer_id'].isin(outflow_count_index)
overdraft_users = outflows.groupby('prism_consumer_id')['Overdraft Count'].sum().apply(lambda x: 1 if x > 1 else 0)
features.append(overdraft_users)
overdraft_users.head()

prism_consumer_id
0    0
2    0
4    0
7    1
9    0
Name: Overdraft Count, dtype: int64

### Cumulative Weighted Default Monthly Average

In [32]:
def add_outflow_ages(outflows):
    outflows = outflows.copy()
    outflows = outflows.merge(cons[['prism_consumer_id', 'evaluation_date']], on='prism_consumer_id', how='outer').dropna()
    
    outflows['posted_month'] = pd.to_datetime(outflows['posted_date']).dt.month
    outflows['posted_year'] = pd.to_datetime(outflows['posted_date']).dt.year
    outflows['posted_date_id'] = 12 * (outflows['posted_year'] - min(outflows['posted_year'])) + outflows['posted_month']

    outflows['eval_month'] = pd.to_datetime(outflows['evaluation_date']).dt.month
    outflows['eval_year'] = pd.to_datetime(outflows['evaluation_date']).dt.year
    outflows['eval_date_id'] = 12 * (outflows['eval_year'] - min(outflows['posted_year'])) + outflows['eval_month']

    outflows['age'] = (outflows['evaluation_date'] - outflows['posted_date']).apply(lambda x: x.days)
    outflows['month_age'] = outflows['eval_date_id'] - outflows['posted_date_id']
    return outflows

In [33]:
def get_cum_weighted_def_val(outflows):
    outflows = add_outflow_ages(outflows)

    overdraft_outflows = (
        outflows[outflows['category_description'] == 'OVERDRAFT']
        [['prism_consumer_id', 'amount', 'age', 'month_age']]
    )

    overdraft_outflows['cum_weighted_def_val'] = (overdraft_outflows['amount'] / (1 + np.power(np.e, (overdraft_outflows['age'] / 30))))

    features_by_month = (
        overdraft_outflows.groupby(['prism_consumer_id','month_age'])
        .sum()
        .reset_index()
        .groupby('prism_consumer_id')[['cum_weighted_def_val']]
        .sum()
        .merge(cons[['prism_consumer_id']], on='prism_consumer_id', how='outer')
        .fillna(0.0)
    )
    return features_by_month

cum_weighted_def_val = get_cum_weighted_def_val(outflows)
features.append(cum_weighted_def_val)
cum_weighted_def_val.head()

,prism_consumer_id,cum_weighted_def_val
0,7,52.632889
1,11,0.293115
2,56,687.738453
3,78,6.637108
4,81,1.559726


### Merge all features into feature matrix

In [34]:
feature_df = reduce(
    lambda left,right: pd.merge(left,right, on='prism_consumer_id', how='outer'), 
    features
)

feature_dates = re.findall(r"\w+_date", str(list(feature_df.columns)))
feature_df['feature_date'] = feature_df[feature_dates].max(axis=1)
feature_df.drop(feature_dates, axis=1, inplace=True)
feature_df.fillna(0.0, inplace=True)
feature_df

,prism_consumer_id,minbalance,maxbalance,stdbalance,total_balance,avg_monthly_spending,num_monthly_purchase,memo_clean,unsufficient_balance,account_count,...,PAYCHECK_prop,PETS_prop,RENT_prop,SELF_TRANSFER_prop,TAX_prop,TRAVEL_prop,UNCATEGORIZED_prop,Overdraft Count,cum_weighted_def_val,feature_date
0,0,6182.60,9907.23,2633.711130,15154.285773,7703.614883,80.915055,0.0,2,2,...,0.000000,0.001800,0.00000,0.169256,0.000000,0.004219,0.031668,0.0,0.000000,2023-04-13
1,2,8079.43,17426.83,6609.609926,1546.065942,9354.337258,98.743961,0.0,2,2,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.096162,0.0,0.000000,2022-02-15
2,4,0.00,4871.39,2644.903088,3377.081389,30563.426238,33.035880,0.0,3,3,...,0.000000,0.000000,0.00000,0.073396,0.000000,0.000000,0.015665,0.0,0.000000,2021-08-13
3,7,517.95,7071.18,3766.231217,7311.445429,23188.278116,138.265235,0.0,3,3,...,0.000000,0.001360,0.00000,0.000000,0.013673,0.021760,0.224591,1.0,52.632889,2021-08-08
4,9,13348.76,13348.76,0.000000,5873.815289,11186.144777,79.267677,0.0,1,1,...,0.000000,0.008849,0.08901,0.005356,0.000000,0.000082,0.111047,0.0,0.000000,2023-04-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,5941,2058.81,2058.81,0.000000,-1816.084426,4052.617937,84.468579,0.0,1,1,...,0.000000,0.000000,0.00000,0.000000,0.000102,0.000000,0.113138,0.0,0.000000,2023-01-06
2974,5943,2431.44,2431.44,0.000000,4121.415493,5769.212382,25.538522,0.0,1,1,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000730,0.078323,0.0,0.000000,2023-02-03
2975,5944,45.74,45.74,0.000000,-765.389130,1987.821739,88.605072,0.0,1,1,...,0.016632,0.000000,0.00000,0.000000,0.000000,0.000000,0.031674,0.0,0.000000,2023-02-01
2976,5945,59.88,59.88,0.000000,-1557.370000,3474.895833,103.541667,0.0,1,1,...,0.000000,0.000000,0.00000,0.000000,0.008394,0.003557,0.188024,0.0,0.000000,2023-02-02


## Final Feature Matrix

In [35]:
sorted_cons = cons.sort_values('evaluation_date')

dropped_cols = ['prism_consumer_id', 'evaluation_date', 'feature_date', 'APPROVED']
feature_matrix = pd.merge(sorted_cons, feature_df, on='prism_consumer_id', how='left')

# Make sure no invalid training data is pulled
assert np.mean(feature_matrix['evaluation_date'] < feature_matrix['feature_date']) == 0, "Features pulled from dates after evaluation_date"
feature_matrix.drop(dropped_cols, axis=1, inplace=True)

## Save Final Feature Matrix

In [37]:
feature_matrix.to_csv(f'{DATA_PATH}/processed/feature_matrix.csv', index=False)